In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 109.6 MB/s eta 0:00:00


In [13]:
import pulp

CARRIERS = [1, 2, 3, 4, 5, 6, 7, 8]
LANES = [1, 2, 3]
ALLIANCE_A = [1, 2, 3, 4]
ALLIANCE_B = [5, 6, 7, 8]

demand = {
    1: 650,  # Lane 1
    2: 775,  # Lane 2
    3: 880   # Lane 3
}

costs_data = [
    [392, 368, 311], # Carrier 1
    [268, 265, 202], # Carrier 2
    [208, 215, 299], # Carrier 3
    [315, 279, 347], # Carrier 4
    [340, 340, 279], # Carrier 5
    [249, 375, 397], # Carrier 6
    [324, 363, 328], # Carrier 7
    [264, 285, 370]  # Carrier 8
]

costs = {i: {j: costs_data[i-1][j-1] for j in LANES} for i in CARRIERS}

# Parameters: Constraints
MIN_MQC = 1000
CARRIER_LANE_CAP_PERCENT = 0.60
ALLIANCE_LANE_CAP_PERCENT = 0.90
BIG_M = sum(demand.values())

## Problem definition
prob = pulp.LpProblem("Shipper_Freight_Allocation", pulp.LpMinimize)

## Decision Variables
# x_ij: Freight (continuous) allocated to carrier i on lane j
x = pulp.LpVariable.dicts("Freight",
                          ((i, j) for i in CARRIERS for j in LANES),
                          lowBound=0,
                          cat='Continuous')

# y_i: 1 if carrier i is used, 0 otherwise (binary)
y = pulp.LpVariable.dicts("CarrierUsed",
                          CARRIERS,
                          cat='Binary')
prob += pulp.lpSum(costs[i][j] * x[(i, j)]
                   for i in CARRIERS
                   for j in LANES), "Total_Transportation_Cost"


# Constraints

for j in LANES:
    prob += pulp.lpSum(x[(i, j)] for i in CARRIERS) == demand[j], f"Demand_Lane_{j}"
for i in CARRIERS:
    for j in LANES:
        prob += x[(i, j)] <= CARRIER_LANE_CAP_PERCENT * demand[j], f"Carrier_Cap_i{i}_j{j}"
for j in LANES:
    # Alliance A
    prob += pulp.lpSum(x[(i, j)] for i in ALLIANCE_A) <= ALLIANCE_LANE_CAP_PERCENT * demand[j], f"Alliance_A_Cap_Lane_{j}"
    # Alliance B
    prob += pulp.lpSum(x[(i, j)] for i in ALLIANCE_B) <= ALLIANCE_LANE_CAP_PERCENT * demand[j], f"Alliance_B_Cap_Lane_{j}"
for i in CARRIERS:
    total_freight_for_carrier = pulp.lpSum(x[(i, j)] for j in LANES)

    # (a) If used (y_i=1), must ship at least MIN_MQC
    prob += total_freight_for_carrier >= MIN_MQC * y[i], f"MQC_Lower_Bound_i{i}"

    # (b) If not used (y_i=0), must ship 0 (Big-M)
    prob += total_freight_for_carrier <= BIG_M * y[i], f"MQC_Upper_Bound_i{i}"


print("Solving the model...")   ## CBC by default
prob.solve()
print(f"\nOptimization Status: {pulp.LpStatus[prob.status]}")

if prob.status == pulp.LpStatusOptimal:
    print(f"\nOptimal Total Cost = ${pulp.value(prob.objective):,.2f}")

    print("\n--- Optimal Allocation Plan ---")
    print("------------------------------------------------------------")
    print(f"{'Carrier':<10} {'Lane 1':<12} {'Lane 2':<12} {'Lane 3':<12} {'Total':<12}")
    print("------------------------------------------------------------")

    lane_totals = {j: 0 for j in LANES}

    for i in CARRIERS:
        total_for_carrier = sum(x[(i, j)].varValue for j in LANES)

        if total_for_carrier > 1e-5:
            lane1_val = x[(i, 1)].varValue
            lane2_val = x[(i, 2)].varValue
            lane3_val = x[(i, 3)].varValue

            lane_totals[1] += lane1_val
            lane_totals[2] += lane2_val
            lane_totals[3] += lane3_val

            print(f"{i:<10} {lane1_val:<12.2f} {lane2_val:<12.2f} {lane3_val:<12.2f} {total_for_carrier:<12.2f}")

    print("------------------------------------------------------------")
    print(f"{'Total':<10} {lane_totals[1]:<12.2f} {lane_totals[2]:<12.2f} {lane_totals[3]:<12.2f} {sum(lane_totals.values()):<12.2f}")
    print(f"{'Demand':<10} {demand[1]:<12.2f} {demand[2]:<12.2f} {demand[3]:<12.2f}")

    print("\n--- Carrier Participation (y_i) ---")
    for i in CARRIERS:
        if y[i].varValue > 0.5:
            print(f"Carrier {i} is USED (y=1)")

else:
    print("The problem did not find an optimal solution.")

Solving the model...

Optimization Status: Optimal

Optimal Total Cost = $621,111.00

--- Optimal Allocation Plan ---
------------------------------------------------------------
Carrier    Lane 1       Lane 2       Lane 3       Total       
------------------------------------------------------------
2          260.00       465.00       528.00       1253.00     
8          390.00       310.00       352.00       1052.00     
------------------------------------------------------------
Total      650.00       775.00       880.00       2305.00     
Demand     650.00       775.00       880.00      

--- Carrier Participation (y_i) ---
Carrier 2 is USED (y=1)
Carrier 8 is USED (y=1)


In [ ]:
|